In [1]:
!rm -rf ./trboard

In [2]:
import os
import tensorflow as tf
import numpy as np
import math

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
DICT_SIZE = 5*10**4
EMBEDDING_SIZE = 64

In [4]:
WINDOW_SIZE = 5

In [5]:
with tf.name_scope('word_embedding'):
    embedding = tf.Variable(
        np.random.uniform(low=-0.05, high=0.05, size=(DICT_SIZE, EMBEDDING_SIZE)).astype(np.float32),
        name='embedding'
    )

In [6]:
embedding.name

'word_embedding/embedding:0'

In [7]:
with tf.name_scope('input'):
    x = tf.placeholder(shape=[None, WINDOW_SIZE-1], dtype=tf.int32)    
    x_embedded = tf.nn.embedding_lookup(embedding, x)
    
with tf.name_scope('y'):
    y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

In [8]:
x

<tf.Tensor 'input/Placeholder:0' shape=(?, 4) dtype=int32>

In [9]:
x_embedded

<tf.Tensor 'input/embedding_lookup:0' shape=(?, 4, 64) dtype=float32>

In [10]:
_x_embedded = tf.reduce_mean(x_embedded, axis=1)

__x_embedded = tf.reshape(x_embedded, shape=(-1, (WINDOW_SIZE-1)*EMBEDDING_SIZE))

In [11]:
print(_x_embedded)
print(__x_embedded)

Tensor("Mean:0", shape=(?, 64), dtype=float32)
Tensor("Reshape:0", shape=(?, 256), dtype=float32)


In [12]:
with tf.name_scope('hidden_layer'):
    out_layer = tf.layers.Dense(
        activation=tf.nn.elu, 
        units=DICT_SIZE,
    )

In [13]:
nce_weights = tf.Variable(
  tf.truncated_normal(
      [DICT_SIZE, EMBEDDING_SIZE],
      stddev=1.0 / math.sqrt(EMBEDDING_SIZE)
  )
)
nce_biases = tf.Variable( tf.zeros(shape=[DICT_SIZE], dtype=tf.float32))

In [14]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(
      tf.nn.nce_loss(
          weights=nce_weights,
          biases=nce_biases,
          labels=y,
          inputs=_x_embedded,
          num_sampled=500,
          num_classes=DICT_SIZE
      )
    )

In [16]:
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=1)
    train_op = optimizer.minimize(loss)

In [17]:
tf.summary.scalar('loss', loss)

merged = tf.summary.merge_all()

In [18]:
from collections import Counter, defaultdict

def create_dict():
    c = Counter(iter_text())
    return {w:i for i,(w,freq) in enumerate(c.most_common(50000-1)) }

def iter_text():
    with open('./../dz4/norm_big_train_agg.csv','r') as f:
        for line in f:
            yield from line.strip().split(',')[3].split()
            
    with open('./../dz4/norm_big_test_agg.csv','r') as f:

        for line in f:
            yield from line.strip().split(',')[3].split()

word_dict = create_dict()

In [19]:
# word_dict

In [20]:
def iter_data(batch_size, window_size=WINDOW_SIZE):
    buffer = []
    batch_x, batch_y = [], []
    slice_size = int(window_size/2)
    for word in iter_text():
        if len(buffer) < window_size:
            buffer.append(word_dict.get(word, len(word_dict)))
        else:
            if not(buffer[slice_size]<=100 or buffer[slice_size]==50000):   
                batch_y.append([buffer[slice_size]])    
                batch_x.append(buffer[:slice_size]+buffer[slice_size+1:])

            buffer.pop(0)
            buffer.append(word_dict.get(word, len(word_dict)))
        
        if len(batch_x)==batch_size:
            yield np.array(batch_x), np.array(batch_y)
            batch_x, batch_y = [], []
            


In [21]:
for _x, _y in iter_data(batch_size=4, window_size=WINDOW_SIZE):
    print(_x)
    print(_y)
    break

[[  30   69   12   13]
 [  12   13 6690  193]
 [  13  507  193  104]
 [ 507 6690  104   43]]
[[5505]
 [ 507]
 [6690]
 [ 193]]


In [22]:
from tensorflow.contrib.tensorboard.plugins import projector

def save_metadata(file):
    with open(file, 'w') as f:
        f.write('obj\tname\n')
        for k,v in word_dict.items():
            f.write('{}\t{}\n'.format( k, v))
        f.write('{}\t{}\n'.format( 50000, '<unk>'))


In [23]:
import time

In [ ]:
log_dir = './trboard'
start_time = time.time()
with tf.Session() as sess:
    saver = tf.train.Saver()
#     saver.restore(sess, os.path.join(log_dir, 'a_model.ckpt'))
    
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    tick = 0
    tf.global_variables_initializer().run()
    for epoch in range(40):
        mean_loss = []
        for i,(batch_x, batch_y) in enumerate(iter_data(batch_size=WINDOW_SIZE*1024, window_size=WINDOW_SIZE)):
            _, _loss, _summary = sess.run(
                [train_op, loss, merged],
                feed_dict={
                    x: batch_x,
                    y: batch_y,
                }
            )
            mean_loss.append(_loss)
            
            summary_writer.add_summary(_summary, tick)
            
            tick+=1
        print(np.array(mean_loss).mean(), time.time()-start_time)
        
    config = projector.ProjectorConfig()
    embed= config.embeddings.add()
    embed.tensor_name = 'word_embedding/embedding:0'
    embed.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(summary_writer, config)
    
    save_metadata(os.path.join(log_dir, embed.metadata_path))
    saver.save(sess, os.path.join(log_dir, 'a_model.ckpt'))


In [31]:
!tensorboard --logdir=./trboard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 0.4.0rc3 at http://MBP-Ruslan-2.dvm.int.ridero.eu:6006 (Press CTRL+C to quit)
^C
